## **Module**

In [1]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [2]:
os.chdir("..")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration'

In [3]:
CANCER_TYPE = "LIHC"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
METHOD = 'deseq2'

* **Load Validation score**

In [4]:
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'test_validation.csv', usecols=col)

* **Condition for Filtering**

In [5]:
filter_cond = (group_score['Silhouette'] > 0.23) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > 85) | (group_score['RNA_RF_F1'] > 85)) & \
              ((group_score['miRNA_ANOVA_F1'] > 85) | (group_score['miRNA_RF_F1'] > 85)) & \
              ((group_score['Methylation_ANOVA_F1'] > 85) | (group_score['Methylation_RF_F1'] > 85))
group_score = group_score[filter_cond]

bestSubgroup = group_score.FILENAME.to_list()

In [ ]:
dea_result = list()
for best_group in bestSubgroup:
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    # DEG Extraction
    deg_list = deg_extract(log_fc=1.5, fdr=0.05,
                   cancer_type=CANCER_TYPE, 
                   sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                   file_name=best_group,
                   method=METHOD)
    
    dea_result.append(deg_list)

In [ ]:
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
dea_combine = [col_rename(dea_combine[index], index) for index in range(len(dea_combine))]
dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)

In [ ]:
dea_combine